# Precisely Boundry Check Demo

Building on the Precisely boundary data info, now we can explore the FS use case:  
*Understand which schools are accessible/enrollable based on parent address.* 

"Parent address" will be turned into a geospatial location (latitude and longitude) via geocoding. We need to prove out the ability to use that location to determine which school enrollment zones the point falls within.

**NOTE**: You may use [nbviewer](https://nbviewer.org/) to view this notebook outside your local Jupyter environment.  

In [1]:
# IMPORTS
import geopandas as gpd
import pandas as pd

import os
import urllib.request
import requests
import shutil
from pathlib import Path
from zipfile import ZipFile

import matplotlib.pyplot as plt
from matplotlib import pyplot

import folium

from shapely.geometry import Point, Polygon

## Data loading and joining

In [2]:
# find the file for school attendance zones
filepath = '/Users/preston.mattox@finalsite.com/Downloads/SCHOOL_BOUNDARIES_USA_202312_SHP_SAMPLE/data/'
file = 'school_attendance_zone_objects_usa_sample.shp'

# load school boundaries
saz = gpd.read_file(filepath+file)

# column names to lowercase
saz.columns= saz.columns.str.lower()

In [3]:
# find the file for attendance zone attributes
file_saz_t = 'school_attendance_zone_tertiary_attributes_usa_sample.csv'
saz_t = pd.read_csv(filepath+file_saz_t)
# saz_t.head()

In [4]:
# join with the attributes data to get low/high grades
saz_base_t = saz.join(saz_t.set_index('obj_id_saz'), on='obj_id', how='inner')
# saz_all_att.head(3)

In [5]:
# now add data about the schools themselves (i.e. the "point data")
file_s_att = 'school_point_primary_attributes_usa_sample.csv'
s_att = pd.read_csv(filepath+file_s_att)
# s_att.head()

In [6]:
# bring it all together into one mega df
s_att = s_att.rename(columns={"low_grade": "low_grade_att", "high_grade": "high_grade_att"}, errors="raise")

saz_all_att = saz_base_t.join(s_att.set_index('obj_id'), on='obj_id_pnt', how='inner')

In [7]:
# focus in on a few columns that we care about
saz_all_att[['obj_id', 'obj_name','geometry','nces_schid', 
             'students', 'teachers', 'pk',  'low_grade', 'high_grade', 'low_grade_att','high_grade_att']].head()

,obj_id,obj_name,geometry,nces_schid,students,teachers,pk,low_grade,high_grade,low_grade_att,high_grade_att
0,16074983,QUAIL HOLLOW ELEMENTARY SCHOOL,"POLYGON ((-111.82347 40.59148, -111.82293 40.5...",490014200768,479.0,25.23,True,PK,5,PK,5
1,16074985,SILVER MESA ELEMENTARY SCHOOL,"POLYGON ((-111.83516 40.59361, -111.83488 40.5...",490014200658,570.0,25.48,False,KG,5,KG,5
2,16074987,MIDVALLEY ELEMENTARY SCHOOL,"POLYGON ((-111.86597 40.62190, -111.86599 40.6...",490014200313,496.0,23.50,False,KG,5,KG,5
3,16075168,ELK RUN ELEMENTARY SCHOOL,"POLYGON ((-112.11084 40.70057, -112.11012 40.7...",490036001412,645.0,26.31,False,KG,6,KG,6
4,16075170,ACADEMY PARK ELEMENTARY SCHOOL,"POLYGON ((-111.98659 40.67595, -111.98660 40.6...",490036000196,376.0,19.00,False,KG,6,KG,6


----

### Carpentry for sorting

Our grade columns are not set up in a way that allows for easy and correct sorting. Let's convert values to categoricals to resolve this.

In [8]:
# look at low grade values
saz_all_att['low_grade'].unique()

array(['PK', 'KG', '07', '06', '10', '09'], dtype=object)

In [9]:
# Define the custom ordering for low grades
custom_order = ['PK', 'KG', '01', '02', '03', '04', '05', '06', '07', '09', '10', '11', '12']

# Convert the low grade column to categorical with the custom ordering
saz_all_att['low_grade'] = pd.Categorical(saz_all_att['low_grade'], categories=custom_order, ordered=True)

In [10]:
# look at high grade values
saz_all_att['high_grade'].unique()

array([ 5,  6,  9,  8, 12])

In [11]:
# TODO: fix high grades so that they sort correctly

# Demo the school attendance zone identification use case

Create a df of latitude & longitude pairs in this area

We can use [this resource](https://developers.google.com/maps/documentation/javascript/examples/event-click-latlng) to find sample points in our region of interest.

In [12]:
# we will later want to join these points with the above gdf
# let's grab the CRS from it and project our new coordinates in this space
target_crs = saz_all_att.crs

In [13]:
# Sample collection of latitude and longitude pairs
lat_lng_pairs = [
    { "lat": 40.5974125741681, "lng": -111.9354563470756 },
    { "lat": 40.6011924027903, "lng": -111.87932307681193 },
    { "lat": 40.61266091930544, "lng": -111.94944674929728 },
    # Add more latitude and longitude pairs as needed
]

# Create a GeoDataFrame
geometry = [Point(lng, lat) for lat, lng in zip([pair['lat'] for pair in lat_lng_pairs], [pair['lng'] for pair in lat_lng_pairs])]
gdf_points = gpd.GeoDataFrame(lat_lng_pairs, geometry=geometry, crs=target_crs)

# Add a new column "category" with value "home" and row index
gdf_points['label'] = 'home_sample_' + gdf_points.index.astype(str)

# Display the GeoDataFrame
gdf_points

,lat,lng,geometry,label
0,40.597413,-111.935456,POINT (-111.93546 40.59741),home_sample_0
1,40.601192,-111.879323,POINT (-111.87932 40.60119),home_sample_1
2,40.612661,-111.949447,POINT (-111.94945 40.61266),home_sample_2


In [14]:
# Create a Folium map centered around the points
m = folium.Map(location=[gdf_points.geometry.y.mean(), gdf_points.geometry.x.mean()], zoom_start=12)

# Add annotated points to the map    
for idx, row in gdf_points.iterrows():
    popup = row['label']
    folium.Marker(location=[row.geometry.y, row.geometry.x], popup=popup).add_to(m)

# Display the map
m

In [15]:
# Perform a spatial join to find points that fall within polygons
gdf_joined = gpd.sjoin(gdf_points, saz_all_att, predicate='within'
                      ).sort_values(by=['label', 'low_grade']).reset_index()

# Display the result
gdf_joined.head()

,index,lat_left,lng,geometry,label,index_right,obj_id,obj_name,obj_typ,obj_subtcd,...,eighth,ninth,tenth,eleventh,twelfth,low_grade_att,high_grade_att,gender,school_typ,ed_level
0,0,40.597413,-111.935456,POINT (-111.93546 40.59741),home_sample_0,141,16075468,RIVERSIDE ELEMENTARY SCHOOL,School,SAZ,...,False,False,False,False,False,KG,6,NaN,R,PM
1,0,40.597413,-111.935456,POINT (-111.93546 40.59741),home_sample_0,127,16075440,WEST JORDAN MIDDLE SCHOOL,School,SAZ,...,True,True,False,False,False,07,9,NaN,R,MH
2,0,40.597413,-111.935456,POINT (-111.93546 40.59741),home_sample_0,143,16075472,WEST JORDAN HIGH SCHOOL,School,SAZ,...,False,False,True,True,True,10,12,NaN,R,H
3,1,40.601192,-111.879323,POINT (-111.87932 40.60119),home_sample_1,2,16074987,MIDVALLEY ELEMENTARY SCHOOL,School,SAZ,...,False,False,False,False,False,KG,5,NaN,R,P
4,1,40.601192,-111.879323,POINT (-111.87932 40.60119),home_sample_1,105,16075399,UNION MIDDLE SCHOOL,School,SAZ,...,True,False,False,False,False,06,8,NaN,R,M


In [16]:
gdf_joined.columns

Index(['index', 'lat_left', 'lng', 'geometry', 'label', 'index_right',
       'obj_id', 'obj_name', 'obj_typ', 'obj_subtcd', 'obj_subtyp', 'country',
       'metro', 'lat_right', 'lon', 'reldate', 'obj_area', 'att_id',
       'obj_id_pnt', 'low_grade', 'high_grade', 'nces_schid', 'nces_disid',
       'grtschl_id', 'choice', 'coextensiv', 'students', 'teachers', 'pk',
       'kg', 'first', 'second', 'third', 'fourth', 'fifth', 'sixth', 'seventh',
       'eighth', 'ninth', 'tenth', 'eleventh', 'twelfth', 'low_grade_att',
       'high_grade_att', 'gender', 'school_typ', 'ed_level'],
      dtype='object')

In [17]:
# view the results that we care about
gdf_joined[['label','geometry','obj_name','low_grade','high_grade']].sort_values(by=['label', 'low_grade'])

,label,geometry,obj_name,low_grade,high_grade
0,home_sample_0,POINT (-111.93546 40.59741),RIVERSIDE ELEMENTARY SCHOOL,KG,6
1,home_sample_0,POINT (-111.93546 40.59741),WEST JORDAN MIDDLE SCHOOL,07,9
2,home_sample_0,POINT (-111.93546 40.59741),WEST JORDAN HIGH SCHOOL,10,12
3,home_sample_1,POINT (-111.87932 40.60119),MIDVALLEY ELEMENTARY SCHOOL,KG,5
4,home_sample_1,POINT (-111.87932 40.60119),UNION MIDDLE SCHOOL,06,8
5,home_sample_1,POINT (-111.87932 40.60119),HILLCREST HIGH SCHOOL,09,12
6,home_sample_2,POINT (-111.94945 40.61266),WEST JORDAN ELEMENTARY SCHOOL,KG,6
7,home_sample_2,POINT (-111.94945 40.61266),WEST JORDAN MIDDLE SCHOOL,07,9
8,home_sample_2,POINT (-111.94945 40.61266),WEST JORDAN HIGH SCHOOL,10,12


### Prep for final plotting

#### Points

In [18]:
# clean up the table of points

# make a copy of the df using only the columns we care about
df = gdf_joined[['label','geometry','obj_name','low_grade','high_grade']].sort_values(
    by=['label', 'low_grade']).copy()

# consolidate all the school info columns into a single column
df['school'] = df.apply(lambda row: ''.join([str(row['obj_name'])
                          ,' [', str(row['low_grade']), ',', str(row['high_grade']),']']), axis=1)

# drop the (now redundant) columns
df = df.drop(columns=['obj_name','low_grade','high_grade'])

# further flatten all schoold into a single entry for simplified point creation later
# note that we use the HTML line break "<br>" as a delimiter because that's what leaflet will regonize
df['school_combined'] = df[['label','geometry','school']].groupby(['label','geometry'])['school'].transform(lambda x: ';<br>'.join(x))
df = df[['label','geometry','school_combined']].drop_duplicates()

gdf_joined_simple = df.copy().reset_index()
gdf_joined_simple

,index,label,geometry,school_combined
0,0,home_sample_0,POINT (-111.93546 40.59741),"RIVERSIDE ELEMENTARY SCHOOL [KG,6];<br>WEST JO..."
1,3,home_sample_1,POINT (-111.87932 40.60119),"MIDVALLEY ELEMENTARY SCHOOL [KG,5];<br>UNION M..."
2,6,home_sample_2,POINT (-111.94945 40.61266),"WEST JORDAN ELEMENTARY SCHOOL [KG,6];<br>WEST ..."


#### Polygons

In [19]:
# collect up the relevant boundary data

# go back to the saz_all_att df and grab only the school names that align with our sample points
names_to_match = gdf_joined['obj_name'].unique()

# Select rows from saz_att_all where "obj_name" matches any value in names_to_match
matching_rows = saz_all_att[saz_all_att['obj_name'].isin(names_to_match)].sort_values(by='obj_name')

# Display the selected rows
matching_rows[['obj_name','low_grade','high_grade','geometry']]

,obj_name,low_grade,high_grade,geometry
94,HILLCREST HIGH SCHOOL,09,12,"POLYGON ((-111.87248 40.63011, -111.87203 40.6..."
2,MIDVALLEY ELEMENTARY SCHOOL,KG,5,"POLYGON ((-111.86597 40.62190, -111.86599 40.6..."
141,RIVERSIDE ELEMENTARY SCHOOL,KG,6,"POLYGON ((-111.92601 40.60880, -111.92527 40.6..."
105,UNION MIDDLE SCHOOL,06,8,"POLYGON ((-111.85871 40.61521, -111.85859 40.6..."
144,WEST JORDAN ELEMENTARY SCHOOL,KG,6,"POLYGON ((-111.94150 40.63127, -111.94116 40.6..."
143,WEST JORDAN HIGH SCHOOL,10,12,"MULTIPOLYGON (((-112.05605 40.63124, -112.0546..."
127,WEST JORDAN MIDDLE SCHOOL,07,9,"POLYGON ((-111.93793 40.63152, -111.93795 40.6..."


# Display the final map

This map should show schools associated with each location (point), as well as the ability to toggle each school attendance zone (polygon) on and off

In [20]:
# plot the results

from folium.plugins import MarkerCluster
import random

# create a base map centered on Salt Lake City, UT
m_final = folium.Map(
    location=[40.6, -111.9],
    zoom_start=12,
)

### SCHOOL ZONES

# Add each polygon to the map
for idx, row in matching_rows.iterrows():
    # generate a random display color
    random_color = '#{:06x}'.format(random.randint(0, 0xFFFFFF))
    # plot the polygon
    folium.GeoJson(row.geometry.__geo_interface__, name=row['obj_name'], show=False,
                   style_function=lambda x, color=random_color: {'fillColor': color, 'color': color}
                  ).add_to(m_final)

### SAMPLE POINTS

# Add annotated points to the map    
for idx, row in gdf_joined_simple.iterrows():
# for idx, row in gdf_joined.sample(1).iterrows():
    folium.Marker(location=[row.geometry.y, row.geometry.x], name=idx,
         popup=folium.Popup("<b>Location:</b> "+gdf_joined_simple.iloc[idx]['label']+"<br>"
                            "<b>Schools:</b> "+gdf_joined_simple.iloc[idx]['school_combined']+"<br>"
            ,max_width=500)
         ).add_to(m_final)

### LAYER CONTROL
folium.LayerControl(collapsed=False).add_to(m_final)

# display the map
m_final